## LIMBO Instrumentation
* requirements:
    * mlib_devel : [m2021a branch](https://github.com/liuweiseu/mlib_devel/tree/m2021a)
    * casperfpga : [py38 branch](https://github.com/liuweiseu/casperfpga/tree/py38)
    * design     : [snap_adc.slx](https://github.com/liuweiseu/snap_hdl_tut/blob/master/snap_adc.slx)

### Step0: Import necessary packages

In [1]:
import os
import sys
import casperfpga
import logging
import time
import redis
import numpy as np
import struct

### Step1: Set parameters

In [4]:
'''
SNAP board info
'''
snap_ip  = '192.168.2.100'
port    = 69


#fpg_file='snap_tut_adc_2023-06-09_1222.fpg'
fpg_file = 'snap_hdl_pfb_v4_2023-08-01_1230.fpg'
'''
Parameters for spectrameter
''' 
# sample freq
fs = 500
#fs = 1000
adc_ref = 10
# adc gain: it's voltage multplier inside the ADC IC.
adc_gain = 4



### Step2: Connect to the SNAP board 

In [3]:
logger=logging.getLogger('snap')
logging.basicConfig(filename='snap.log',level=logging.DEBUG)
snap=casperfpga.CasperFpga(snap_ip, port, logger=logger)

ERROR:tornado.application:Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOLoop object at 0x7f9f847bccd0>>, <Future finished exception=KatcpClientError("Failed to connect to ('192.168.2.100', 69)")>)
Traceback (most recent call last):
  File "/home/wei/.conda/envs/py38/lib/python3.8/site-packages/tornado/ioloop.py", line 740, in _run_callback
    ret = callback()
  File "/home/wei/.conda/envs/py38/lib/python3.8/site-packages/tornado/ioloop.py", line 764, in _discard_future_result
    future.result()
  File "/home/wei/.conda/envs/py38/lib/python3.8/site-packages/tornado/gen.py", line 782, in run
    yielded = self.gen.send(value)
  File "/home/wei/.conda/envs/py38/lib/python3.8/site-packages/katcp/client.py", line 682, in _install
    raise KatcpClientError("Failed to connect to {0!r}"
katcp.core.KatcpClientError: Failed to connect to ('192.168.2.100', 69)


### Step4: Upload fpg file

In [5]:
fpg = '../fpg/'+fpg_file
print('fpg file: ',fpg)
snap.upload_to_ram_and_program(fpg)
# We should get system info in "upload_to_ran_and_program", but it seems there are some issues in the casperfpga
snap.get_system_information(fpg,initialise_objects=False)

fpg file:  ../fpg/snap_hdl_pfb_v4_2023-08-01_1230.fpg


### Step5: Init clk and adc

In [7]:
snap.adcs

['snap_adc']

In [8]:
# numChannel depends on fs
if(fs==1000):
    numChannel = 1
    inputs = [1,1,1,1]
elif(fs==500):
    numChannel = 2
    inputs = [1,1,3,3]
# init adc and clk
adc=snap.adcs['snap_adc']
adc.ref = adc_ref
adc.selectADC()
adc.init(sample_rate=fs,numChannel=numChannel)
adc.rampTest(retry=True)
adc.adc.selectInput(inputs)
adc.selectADC()
adc.set_gain(adc_gain)

File in flash is:  snap_hdl_pfb_v4_2023-08-01_1230.fpg
File in flash is:  snap_hdl_pfb_v4_2023-08-01_1230.fpg


In [9]:
snap.listdev()

['acc_len',
 'adc16_controller',
 'adc16_use_synth',
 'adc16_wb_ram0',
 'adc16_wb_ram1',
 'adc16_wb_ram2',
 'led_ctrl',
 'lmx_ctrl',
 'pfb_status',
 'pwr_snapshot_ss0_bram',
 'pwr_snapshot_ss0_ctrl',
 'pwr_snapshot_ss0_status',
 'pwr_snapshot_ss1_bram',
 'pwr_snapshot_ss1_ctrl',
 'pwr_snapshot_ss1_status',
 'sw_trig',
 'sys',
 'sys_board_id',
 'sys_clkcounter',
 'sys_ctl',
 'sys_rev',
 'sys_rev_rcs',
 'sys_scratchpad',
 'ten_gbe']

In [28]:
snap.registers

['adc_snap_ctrl', 'adc_snap_status', 'adc_trig', 'led_ctrl', 'sys_board_id', 'sys_rev', 'sys_rev_rcs', 'sys_scratchpad', 'sys_clkcounter']

In [51]:
d=adc.readRAM()
print(type(d))

<class 'dict'>


In [52]:
d=np.array(d[1])

In [53]:
d.shape

(128, 8)

In [81]:
a=np.array([[1,2,3,4],[5,6,7,8]])
print(a)


[[1 2 3 4]
 [5 6 7 8]]


In [82]:
a=a[:,[3,2,1,0]]
print(a)

[[4 3 2 1]
 [8 7 6 5]]


In [83]:
a.reshape(4,2)

array([[4, 3],
       [2, 1],
       [8, 7],
       [6, 5]])

In [84]:
b=np.array([[1,2,3,4],[5,6,7,8]])
b.reshape(4,2)

array([[1, 2],
       [3, 4],
       [5, 6],
       [7, 8]])